# Label Conversion
This file is used to convert RetinaFace output label to a standardized format.

In [ ]:
import pandas as pd
import json
import os
import cv2

import os


# Standard Libraries
import math
from PIL import Image 
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import pandas as pd


import cv2
import matplotlib.pyplot as plt

### convert retinaface to yolo format

In [ ]:

def convert_retinaface_to_yolo(input_folder, output_folder, image_folder, face_class_id=99):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if not file_name.endswith('.json'):
            continue

        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name.replace('.json', '.txt'))

        with open(input_path, 'r') as f:
            retinaface_output = json.load(f)

        # Handle case where no faces are detected
        if "message" in retinaface_output and retinaface_output["message"] == "No faces detected":
            # Create an empty YOLO file
            open(output_path, 'w').close()
            print(f"No faces detected for {file_name}. Created empty file.")
            continue

        # Get corresponding image dimensions
        image_name = file_name.replace('.json', '.png')
        image_path = os.path.join(image_folder, image_name)

        if not os.path.exists(image_path):
            print(f"Image file {image_path} not found. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image {image_path}. Skipping.")
            continue

        image_height, image_width, _ = image.shape

        yolo_annotations = []

        # Process detected faces
        for face_id, face_data in retinaface_output.items():
            if face_id == "message":
                continue

            score = face_data["score"]
            x_min, y_min, x_max, y_max = face_data["facial_area"]

            # Calculate YOLO format values
            x_center = round(((x_min + x_max) / 2) / image_width, 5)
            y_center = round(((y_min + y_max) / 2) / image_height, 5)
            width = round((x_max - x_min) / image_width, 5)
            height = round((y_max - y_min) / image_height, 5)

            # Append to YOLO annotations
            yolo_annotations.append([face_class_id, x_center, y_center, width, height, score])

        # Save to YOLO annotation file
        if yolo_annotations:
            yolo_df = pd.DataFrame(yolo_annotations, columns=["class", "x_center", "y_center", "width", "height", "confidence"])
            yolo_df.to_csv(output_path, sep=" ", header=False, index=False)
        else:
            # Create an empty file if no annotations exist
            open(output_path, 'w').close()

        print(f"Processed {file_name}. YOLO annotations saved to {output_path}.")



In [ ]:
sampleFolder = "../ssic_image-corpus/scripts/baseline_retinaface/retina_base/"
outputFolder = "../ssic_image-corpus/data_phase-3_randomization/retinaface_output_toyolo/"


imageFolder = "../ssic_image-corpus/data_phase-3_randomization/randomization_unsorted_imageonly/"

convert_retinaface_to_yolo(
    input_folder=sampleFolder,
    output_folder=outputFolder,
    image_folder=imageFolder,
    face_class_id=99)
